# Rossman dataset 

Fuente xgboost: https://xgboost.readthedocs.io/en/release_3.0.0/

https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.ml.ensemble.XGBRegressor

Correr esto:

!pip install xgboost==1.7.6 scikit-learn==1.2.2

https://www.kaggle.com/competitions/rossmann-store-sales/data

- Id - an Id that represents a (Store, Date) duple within the test set
- Store - a unique Id for each store
- Sales - the turnover for any given day (this is what you are predicting)
- Customers - the number of customers on a given day
- Open - an indicator for whether the store was open: 0 = closed, 1 = open
- StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
- SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
- StoreType - differentiates between 4 different store models: a, b, c, d
- Assortment - describes an assortment level: a = basic, b = extra, c = extended
- CompetitionDistance - distance in meters to the nearest competitor store
- CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
- Promo - indicates whether a store is running a promo on that day
- Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
- PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor, plot_importance
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# Carga de datos
train = pd.read_csv('rossman.csv')
stores = pd.read_csv('store.csv')

# Merge de ambos datasets
df = pd.merge(train, stores, on='Store')

# Convertir la columna de fecha
df['Date'] = pd.to_datetime(df['Date'])

# Ordenar por fecha para evitar leakage
df = df.sort_values('Date')

# Filtrar solo tiendas abiertas
df = df.query('Open == 1')

# Variables temporales útiles, mes y dia del mes
df['month'] = df['Date'].dt.month
df['day_of_month'] = df['Date'].dt.day
df['week_of_year'] = df['Date'].dt.isocalendar().week.astype(int)

# Eliminar columnas que no usaremos
df = df.drop(['Date','Customers','Open', 'Store'], axis=1)
df.head()

/var/folders/60/rl4yk8jj3453bx7hbt074rbc0000gn/T/ipykernel_52035/368709671.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('rossman.csv')


,DayOfWeek,Sales,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,month,day_of_month,week_of_year
1016447,2,3139,0,a,1,b,b,900.0,NaN,NaN,1,14.0,2013.0,"Feb,May,Aug,Nov",1,1,1
1016429,2,2401,0,a,1,b,a,90.0,NaN,NaN,1,31.0,2013.0,"Jan,Apr,Jul,Oct",1,1,1
1016606,2,2646,0,a,1,b,b,590.0,NaN,NaN,1,5.0,2013.0,"Mar,Jun,Sept,Dec",1,1,1
1016588,2,3113,0,a,1,b,a,1260.0,6.0,2011.0,0,NaN,NaN,NaN,1,1,1
1016624,2,2907,0,a,1,a,c,18160.0,NaN,NaN,0,NaN,NaN,NaN,1,1,1


In [2]:
train_treshold = int(len(df) * 0.9)

In [3]:
# split 90% de datos
train_df = df[:train_treshold]
test_df = df[train_treshold:]

# Obten X_train, y_train, X_test, y_test
target = 'Sales'
X_train = train_df.drop(target, axis=1)
X_test = test_df.drop(target, axis=1)

y_train = train_df[target]
y_test = test_df[target]

# Codificar variables categóricas (después del split para evitar leakage)
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Alinear columnas en test con train
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

In [4]:
# Xgboost
model = XGBRegressor(n_estimators = 100, max_depth = 3, tree_method = 'hist')

# Fit
model.fit(X_train, y_train)

# Predicción y evaluación
predicciones = model.predict(X_test)
r2_score(y_test, predicciones)

0.502098560333252

In [5]:
# Crea una regresion lineal para ver que tan bien da
from sklearn.linear_model import LinearRegression

# Calcular medias solo en train
means = X_train.mean()

# Aplicar a ambos
X_train = X_train.fillna(means)
X_test = X_test.fillna(means)

# Entrenar modelo
lin_reg = LinearRegression().fit(X_train, y_train)

# Predecir
predicciones = lin_reg.predict(X_test)

# Evaluar
r2_score(y_test, predicciones)

0.20040131916041515

### Tuneando xgboost

- n_estimators: número total de árboles a entrenar. Si usas early_stopping_rounds, puedes poner un número alto sin preocuparte por overfitting.

- max_depth: profundidad máxima de cada árbol. Profundidades mayores capturan más complejidad, pero pueden sobreajustar.

- learning_rate: qué tan rápido aprende el modelo. Valores más bajos requieren más árboles (n_estimators más alto), pero suelen generalizar mejor.

- tree_method: método para construir árboles:

    'exact': busca el mejor split evaluando todos los valores.
    
    'hist': agrupa en bins (default moderno), mucho más rápido.
    
    'gpu_hist': como 'hist' pero usando GPU (⚡ si tienes una buena GPU).

- subsample: fracción de los datos usados en cada árbol (filas). Ayuda a reducir overfitting. Ej: 0.8 usa el 80% de los datos en cada iteración.

- colsample_bytree (¡ojo con el nombre!): fracción de columnas seleccionadas aleatoriamente para cada árbol. También ayuda contra el overfitting.

- early_stopping_rounds: si la métrica en el conjunto de validación no mejora durante N iteraciones, se detiene el entrenamiento (requiere eval_set).

- eval_metric: métrica usada para evaluar el desempeño durante entrenamiento (ej. 'rmse', 'mae', 'logloss', 'auc'). Es la que define si se activa early_stopping.

- gamma: mínima ganancia necesaria para hacer un split adicional. Ayuda a regular complejidad.

- reg_lambda: regularización L2 (Ridge). Útil para evitar sobreajuste.

- reg_alpha: regularización L1 (Lasso). Puede hacer pruning de features.



In [6]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

model = XGBRegressor(
    n_estimators=10_000,
    max_depth=5,
    learning_rate=0.1,  # Corregido aquí
    reg_lambda=0.1,
    subsample=0.5,
    colsample_bytree=0.8,  # El parámetro correcto es "colsample_bytree"
    tree_method='hist',
    early_stopping_rounds=20,
)

model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=100)

[0]	validation_0-rmse:3063.10132	validation_1-rmse:3049.03123
[100]	validation_0-rmse:2016.82554	validation_1-rmse:2123.00886
[200]	validation_0-rmse:1698.19713	validation_1-rmse:1818.29351
[300]	validation_0-rmse:1502.31513	validation_1-rmse:1629.53973
[400]	validation_0-rmse:1373.41342	validation_1-rmse:1506.14211
[500]	validation_0-rmse:1282.75441	validation_1-rmse:1423.09701
[600]	validation_0-rmse:1216.55547	validation_1-rmse:1365.02913
[700]	validation_0-rmse:1162.17347	validation_1-rmse:1317.47826
[800]	validation_0-rmse:1120.22066	validation_1-rmse:1284.94723
[900]	validation_0-rmse:1089.02850	validation_1-rmse:1265.53446
[1000]	validation_0-rmse:1061.21622	validation_1-rmse:1245.91451
[1065]	validation_0-rmse:1046.48643	validation_1-rmse:1236.42515


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=20,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=10000,
             n_jobs=None, num_parallel_tree=None, ...)

In [7]:
# Grafica ambos errores 


In [8]:
# Calibracion

### Feature importances 

In [9]:
#

```
from sklearn.inspection import PartialDependenceDisplay

for feature in importancias.features:
    PartialDependenceDisplay.from_estimator(model, X_test, [feature])
    plt.show()
```

In [10]:
# Partial dependence plot 


# Feature importance

### Gain 
Gain es que tanto disminuye la función de perdida, en promedio al usar cada feature


In [11]:
from xgboost import plot_importance

# Grafica la importancia, por importance_type y el max_num_features


### Weight
Weighr es el porcentaje de veces que se uso esa variable